In [1]:
import pandas as pd
from pathlib import Path

BASE = Path("../data/clean/vct_2024")
DIM = Path("../data/gold/dimensions")
OUT = Path("../data/gold/facts")

OUT.mkdir(parents=True, exist_ok=True)

In [13]:
# Silver source
players_stats = pd.read_parquet(
    BASE / "players_stats/players_stats.parquet"
)

# Dimensions
dim_player = pd.read_parquet(DIM / "dim_player.parquet")
dim_team = pd.read_parquet(DIM / "dim_team.parquet")
dim_tournament = pd.read_parquet(DIM / "dim_tournament.parquet")
dim_agent = pd.read_parquet(DIM / "dim_agent.parquet")

In [16]:
mask_aggregate = players_stats["agent_name"].str.contains(",")

In [17]:
print(len(players_stats))
print(len(players_stats[mask_aggregate]))

11315
3224


In [18]:
players_stats = players_stats[
    ~mask_aggregate
].copy()

In [19]:
print(len(players_stats))

8091


In [ ]:
players_stats = players_stats[
    (players_stats["stage_name"] != "All Stages") &
    (players_stats["match_type_name"] != "All Match Types")
].copy()

In [21]:
print(len(players_stats))

8091


In [22]:
fact = players_stats.merge(
    dim_player,
    on="player_name",
    how="left"
)

In [24]:
print(fact["player_id"].isna().sum())

0


In [26]:
fact = fact.merge(
    dim_team,
    on="team_name",
    how="left"
)

In [27]:
print(fact["team_id"].isna().sum())

0


In [28]:
fact = fact.merge(
    dim_agent,
    on="agent_name",
    how="left"
)

In [29]:
print(fact["agent_id"].isna().sum())

0


In [30]:
fact = fact.merge(
    dim_tournament,
    on=[
        "tournament_name",
        "stage_name",
        "match_type_name"
    ],
    how="left"
)

In [31]:
print(fact["tournament_id"].isna().sum())

0


In [32]:
len(fact.columns)

32

In [33]:
fact_player_competition_stats = fact[
    [
        # Keys
        "player_id",
        "team_id",
        "agent_id",

        "tournament_id",
        "stage_id",
        "match_type_id",

        # Volume
        "rounds_played",

        # Ratings
        "rating",
        "average_combat_score",

        # Ratios
        "kills_deaths_ratio",
        "kills_per_round",
        "assists_per_round",
        "first_kills_per_round",
        "first_deaths_per_round",

        # Percentages
        "kill_assist_trade_survive_percentage",
        "average_damage_per_round",
        "headshot_percentage",
        "clutch_success_percentage",

        # Totals
        "maximum_kills_single_map",
        "kills",
        "deaths",
        "assists",
        "first_kills",
        "first_deaths",

        # Clutches
        "clutches_won",
        "clutches_played",
    ]
].copy()


In [34]:
len(fact_player_competition_stats.columns)

26

In [39]:
fact_player_competition_stats.head()

,player_id,team_id,agent_id,tournament_id,stage_id,match_type_id,rounds_played,rating,average_combat_score,kills_deaths_ratio,...,headshot_percentage,clutch_success_percentage,maximum_kills_single_map,kills,deaths,assists,first_kills,first_deaths,clutches_won,clutches_played
0,1144,1001,1,2097,4131,26235,26,0.75,138,0.58,...,0.21,0.22,11,11,19,14,1,2,2,9
1,1144,1001,8,2097,4131,26235,16,1.10,185,1.38,...,0.21,0.00,11,11,8,3,2,0,0,0
2,29873,1001,6,2097,4131,26235,26,0.85,203,0.86,...,0.34,0.00,18,18,21,3,2,4,0,0
3,29873,1001,14,2097,4131,26235,16,1.36,169,2.40,...,0.28,0.50,12,12,5,2,1,0,1,2
4,21328,1001,9,2097,4131,26235,26,0.74,92,0.59,...,0.53,0.00,10,10,17,5,0,2,0,2


In [40]:
out_path = OUT / "fact_player_competition_stats.parquet"

fact_player_competition_stats.to_parquet(
    out_path,
    index=False
)